In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData, Table, select, Column, Numeric, Integer, BigInteger, String, DateTime
from datetime import date
import sys

In [2]:
print 'Python version ' + sys.version
print 'Pandas version ' + pd.__version__
print 'SQLalchemy version: ' + sqlalchemy.__version__

Python version 2.7.5 |Anaconda 2.3.0 (64-bit)| (default, Jul  1 2013, 12:37:52) [MSC v.1500 64 bit (AMD64)]
Pandas version 0.16.2
SQLalchemy version: 1.0.5


> We will be using sqlalchemy to connect to a sqllite database. The beauty of using sqlalchemy is that you can reuse the code below and connect to a different database like mssql, mysql, postgresql, or oracle.

# How to add records to table?

In [3]:
def add_records(tbl, name, price, dateadded):
    """ add records to db """
    
    # Create the connection
    engine = create_engine('sqlite:///tutorial.db')
    conn = engine.connect()
    
    # Required for querying tables
    metadata = MetaData(conn)    


    tbl = Table(tbl, metadata,
            Column('id', Integer, primary_key=True),
            Column('name', String(300), nullable=False),  
            Column('price', Numeric(18, 2), nullable=False),
            Column('dateadded', DateTime, nullable=False)    
    )   
    
    # checkfirst=True makes sure to only create if it doesn't exist
    tbl.create(checkfirst=True)
    
    
    sql = tbl.insert()
    
    try:
        sql.execute({'name': name, 
                   'price': price,
                   'dateadded': dateadded})
        
        result = True
            
    except Exception as e:
        print e
        result = False
    
    # Close connection
    conn.close()    

    return result


# How to delete records from a table?

In [4]:
def del_records(tbl, idnum):
    """ delete records from db. Use the primary key, id """
    
    # Create the connection
    engine = create_engine('sqlite:///tutorial.db')
    conn = engine.connect()
    
    # Required for querying tables
    metadata = MetaData(conn)         

    try:
        # Table to query
        tbl = Table(tbl, metadata, autoload=True)    
        
        # sql code
        sql = tbl.delete(tbl.c.id == idnum)
    
        # run sql code
        conn.execute(sql)   
        result = True

    except Exception as e:
        result = False        
    
    # Close connection
    conn.close()      
    
    return result

# How to select records from a table? 

In [5]:
def get_records(tbl, idnum=None):
    """ get records from db. Use the primary key, id """
    
    # Create the connection
    engine = create_engine('sqlite:///tutorial.db')
    conn = engine.connect()
    
    # Required for querying tables
    metadata = MetaData(conn)  

    try:
        # Table to query
        table = Table(tbl, metadata, autoload=True)
        
        # sql code
        if idnum == None:
            sql = table.select()  
        else: 
            sql = table.select(table.c.id == idnum) 
    
        # run sql code
        result = conn.execute(sql)   

        # Insert to a dataframe
        df = pd.DataFrame(data=list(result), columns=result.keys())

    except Exception as e:
        print e
        df = False        

    # Close connection
    conn.close()      
    
    return df

# How to drop a table?

In [6]:
def drop_tbl(tblname):
    """ drop table from db. """
    """ Parameter = name of table """
    
    # Create the connection
    engine = create_engine('sqlite:///tutorial.db')
    conn = engine.connect()
    
    # Required for querying tables
    metadata = MetaData(conn)        
    
    try:    
        # Table to query
        tbl = Table(tblname, metadata, autoload=True)    
    
        # drop table
        tbl.drop(checkfirst=True)  
        result = True

    except Exception as e:
        result = False      
    
    # Close connection
    conn.close()      
    
    return result 

# How to update a table?

In [7]:
def update_records(tbl, idnum, col, val):
    """ update records from db. Use the primary key, id """
    
    # Create the connection
    engine = create_engine('sqlite:///tutorial.db')
    conn = engine.connect()
    
    # Required for querying tables
    metadata = MetaData(conn)         

    try:
        # Table to query
        tbl = Table(tbl, metadata, autoload=True)    
        
        # sql code
        sql = tbl.update().where(tbl.c.id == idnum).values(**{col: val})
    
        # run sql code
        conn.execute(sql)   
        result = True

    except Exception as e:
        print e
        result = False        
    
    # Close connection
    conn.close()      
    
    return result

# How to query for tables in the database?

In [8]:
def get_tbl_names():
    """ get table names from db """
    
    # Create the connection
    engine = create_engine('sqlite:///tutorial.db')
    conn = engine.connect()
    
    # Required for querying tables
    m = MetaData(conn)
    m.reflect(engine)
    for table in m.tables.values():
        print 'TABLE'
        print '-------------'
        print(table.name)
        for column in table.c:
            print(column.name)     

    # Close connection
    conn.close()      
    
    return None

# Below we use the functions created above  

* Where is the tutorial.db located? You will find the file in the same folder of this notebook.

In [9]:
# Lets add some data to a table called "tbl1"

## adding record #1
add_records('tbl1',
            'rateplan',
            '1', 
            date.today())

## adding record #2
add_records('tbl1',
            'rateplan',
            '1', 
            date(2001, 1, 1))

## adding record #3
add_records('tbl1',
            'rateplan',
            '1', 
            date(2014, 12, 31))

True

In [20]:
# select all data from tbl1
get_records('tbl1')

tbl1


False

In [11]:
# select the record where id=1
get_records('tbl1', 1)

,id,name,price,dateadded
0,1,rateplan,1.00,2015-09-03


In [12]:
# select the record where id=3
get_records('tbl1', 3)

,id,name,price,dateadded
0,3,rateplan,1.00,2014-12-31


In [13]:
# update price where id=2
update_records('tbl1', 2, 'price', 50.25)

True

In [14]:
# select the record where id=2
get_records('tbl1', 2)

,id,name,price,dateadded
0,2,rateplan,50.25,2001-01-01


In [15]:
# delete row where id=2
del_records('tbl1', 2)

True

In [16]:
# select all data from tbl1
get_records('tbl1')

,id,name,price,dateadded
0,1,rateplan,1.00,2015-09-03
1,3,rateplan,1.00,2014-12-31


In [17]:
# query tables in DB
get_tbl_names()

TABLE
-------------
tbl1
id
name
price
dateadded


In [18]:
# drop tbl1
drop_tbl('tbl1')

True

In [19]:
# query tables in DB
get_tbl_names()

**Author:** [HEDARO](http://www.hedaro.com)  